In [18]:
#!pip install python-docx
#!pip install openpyxl

In [9]:
from transformers import pipeline
import nltk

In [10]:
tokenizer_en = nltk.data.load('tokenizers/punkt/english.pickle')
tokenizer_fr = nltk.data.load('tokenizers/punkt/french.pickle')
tokenizer_tr = nltk.data.load('tokenizers/punkt/turkish.pickle')

In [11]:
translator_en_ru = pipeline(task='translation', model='Helsinki-NLP/opus-mt-en-ru', device='cuda:0')
translator_fr_ru = pipeline(task='translation', model='Helsinki-NLP/opus-mt-fr-ru', device='cuda:0')
translator_tr_en = pipeline(task='translation', model='Helsinki-NLP/opus-mt-tr-en', device='cuda:0')

In [16]:
#translator_zh_en = pipeline(task='translation', model='Helsinki-NLP/opus-mt-zh-en', device='cuda:0')

# Работа с DOCX

In [7]:
import docx

In [8]:
def copy_runformat(run, etalon_run):
    '''Копирует формат (шрифт, стиль, начертание текста) структуры run из эталона'''
    run.style = etalon_run.style
    run.font.name = etalon_run.font.name
    run.font.size = etalon_run.font.size
    run.font.color.rgb = etalon_run.font.color.rgb
    run.bold = etalon_run.bold
    run.italic = etalon_run.italic
    run.underline = etalon_run.underline

In [9]:
def do_paragraph(paragraph, lang='English'):
    '''Делит абзац на предложения, переводит их и записывает в исходный абзац вместо оригинального текста
    При этом для измененного абзаца выставляется формат (шрифт, стиль, начертание текста)
    первой структуры run оригинального абзаца'''
    trans_phrases = []
    if lang == 'English':
        tokenizer = tokenizer_en
        translator = translator_en_ru
    if lang == 'French':
        tokenizer = tokenizer_fr
        translator = translator_fr_ru
    if lang == 'Turkish':
        tokenizer = tokenizer_tr
        translator = translator_tr_en
#    if lang == 'Chinese':
    phrases = tokenizer.tokenize(paragraph.text)
#    print(phrases)
    if not phrases:
        return
    trans_phrases_dict = translator(phrases)
    for dict in trans_phrases_dict:
        trans_phrases+=list(dict.values())
#    print(trans_phrases)
    if len(paragraph.runs) > 0:
        run = paragraph.runs[0]
        paragraph.text = ' '.join(trans_phrases)
        copy_runformat(paragraph.runs[0], run)
    else:
        paragraph.text = ' '.join(trans_phrases)

In [10]:
def translate_docx(filename, lang='English'):
    '''Переводит входящий файл по абзацам и создает новый файл с переведенным текстом.
    Выходной файл имеет "_res" в конце имени файла'''
    doc = docx.Document(filename)
    for paragraph in doc.paragraphs:
        do_paragraph(paragraph, lang)
        if lang == 'Turkish' or lang == 'Chinese':
            do_paragraph(paragraph)
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for paragraph in cell.paragraphs:
                    do_paragraph(paragraph, lang)
                    if lang == 'Turkish' or lang == 'Chinese':
                        do_paragraph(paragraph)
  
    filename_list = filename.split('.')
    filename_list[-2]+='_res'
    filename_out = '.'.join(filename_list)
    doc.save(filename_out)

In [652]:
translate_docx('test_en.docx')

In [653]:
translate_docx('test_fr.docx', 'French')

In [660]:
translate_docx('test_tr.docx', 'Turkish')

# Работа с XLSX

In [12]:
from openpyxl import load_workbook

In [13]:
def do_cell(cell_value, lang='English'):
    '''Делит текст ячейкки на предложения, переводит их и записывает в исходную ячейку вместо оригинального текста
    При этом сохраняется изначальное форматирование ячейки'''
    trans_phrases = []
    if lang == 'English':
        tokenizer = tokenizer_en
        translator = translator_en_ru
    if lang == 'French':
        tokenizer = tokenizer_fr
        translator = translator_fr_ru
    if lang == 'Turkish':
        tokenizer = tokenizer_tr
        translator = translator_tr_en
#    if lang == 'Chinese':
        
    phrases = tokenizer.tokenize(cell_value)
    if not phrases:
        return
    trans_phrases_dict = translator(phrases)
    for dict in trans_phrases_dict:
        trans_phrases+=list(dict.values())
    return ' '.join(trans_phrases)

In [14]:
def translate_xlsx(filename, lang='English'):
    '''Переводит входящий файл по ячейкам и создает новый файл с переведенным текстом.
    Выходной файл имеет "_res" в конце имени файла '''
    workbook = load_workbook(filename)
    for sheetname in workbook.sheetnames:
        sheet = workbook[sheetname]
        for row in sheet.rows:
            for cell in row:
                if cell.value and cell.data_type=='s' and cell.value[0]!='=':
                    cell.value = do_cell(cell.value, lang)
                    if lang == 'Turkish':
                        cell.value = do_cell(cell.value)
    
    filename_list = filename.split('.')
    filename_list[-2]+='_res'
    filename_out = '.'.join(filename_list)
    workbook.save(filename_out)

In [15]:
translate_xlsx('test_en.xlsx')